## start

In [0]:
from google.colab import files
from google.colab import drive
drive.mount('/content/mount')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mount


In [0]:
import os
print(os.environ['PYTHONPATH'] )

/env/python


In [0]:
import tensorflow
print(tensorflow.__version__)

2.2.0


##dependency

In [0]:
!git clone  https://github.com/yuslzq/transformer.git


Cloning into 'transformer'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 207 (delta 37), reused 122 (delta 27), pack-reused 69
Receiving objects: 100% (207/207), 308.44 KiB | 1.16 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [0]:
!cp -r '/content/mount/My Drive/00_work/train_dt' /content/

In [0]:
!pip install tf-nightly-gpu

     |████████████████████████████████| 523.3MB 32kB/s 
     |████████████████████████████████| 3.0MB 37.9MB/s 
     |████████████████████████████████| 460kB 43.3MB/s 


## Training

In [0]:
%cd /content/transformer
PARAM_SET='base'
HOME='/content/00_work'
DATA_DIR='/content/train_dt'
MODEL_DIR='/content/mount/My\ Drive/00_work/model_base'
VOCAB_FILE=DATA_DIR+'/vocab.ende.32768'
!python transformer_main.py --data_dir=$DATA_DIR --model_dir=$MODEL_DIR \
    --vocab_file=$VOCAB_FILE --param_set=$PARAM_SET \
    --train_steps=100000 --steps_between_evals=4000 \
    --batch_size=8192 --max_length=64 \
    --bleu_source=$DATA_DIR/valu_ja.csv \
    --bleu_ref=$DATA_DIR/valu_en.csv \
    --enable_time_history=false

/content/transformer
2020-05-21 03:29:56.692765: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-21 03:30:00.358257: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-21 03:30:00.414824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-21 03:30:00.415792: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1544] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-21 03:30:00.415848: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-21 03:30:00.678420: I tensorfl

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
!nvcc -v 

## Export

# make pb file

In [0]:
import os
os.environ['PYTHONPATH'] = '/env/python:/content/sys_oper/00_src_models/research/:/content/sys_oper/00_src_models/research/slim/'


In [0]:

PIPELINE_FNAME = '/content/sys_oper/01_configfile/ssd_resnet50_v1_fpn_shared_box_predictor_gt.config'
MODEL_DIR = '/content/mount/My\ Drive/00_work/ssd_ysl_ssd_resnet50_sysoper_gtgt2'
CONFIG_FILE='/content/sys_oper/01_configfile/ssd_resnet50_v1_fpn_shared_box_predictor_gt.config'
EXPORT_OUTPUT_DIR='/content/mount/My\ Drive/00_work/ssd_ysl_ssd_resnet50_sysoper_gtgt2/tflite'
CHECKPOINT_PATH='/content/mount/My\ Drive/00_work/ssd_ysl_ssd_resnet50_sysoper_gtgt2/model.ckpt-11106'
!python /content/sys_oper/00_src_models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$CONFIG_FILE \
--trained_checkpoint_prefix=$CHECKPOINT_PATH \
--output_directory=$EXPORT_OUTPUT_DIR \
--add_postprocessing_op=true

#make tflite file

In [0]:
PATH_TO_MODEL='/content/mount/My\ Drive/00_work/ssd_ysl_ssd_resnet50_sysoper_gtgt2/tflite'
!tflite_convert    \
 --graph_def_file={PATH_TO_MODEL}/tflite_graph.pb \
 --output_file={PATH_TO_MODEL}/detectfpn300.tflite \
 --input_shapes=1,800,800,3 \
 --output_format=TFLITE \
 --input_arrays=normalized_input_image_tensor \
 --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
 --inference_type=FLOAT \
 --mean_values=128 \
 --std_dev_values=128 \
 --change_concat_input_ranges=false \
 --allow_custom_ops

In [0]:
import numpy as np
import shutil

In [0]:
output_directory = './fine_tuned_model'

if os.path.exists(output_directory):
    shutil.rmtree(output_directory)
    
lst = os.listdir(MODEL_DIR)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps = np.array([int(re.findall('\d+',l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(MODEL_DIR, last_model)
print(last_model_path)

!python /content/Finetune_SSD_MobileNetV2/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [0]:
log = os.listdir(MODEL_DIR)
log = [l for l in log if 'events.out.tfevents.' in l]

for item in log:
    print(output_directory)
    shutil.copy(os.path.join(MODEL_DIR, item), output_directory)

./fine_tuned_model


In [0]:
DRIVE_WORKSPACE = '/content/mount/My\ Drive/DogCatDetection'
LATEST_CHECKPOINT = '/content/mount/My\ Drive/DogCatDetection/last_checkpoint'
FOLDER_CHECKPOINT = '/content/mount/My\ Drive/DogCatDetection/'+'train_' # setting number here
# Function to back up all checkpoint: Folder train + number
arr = []
!ls {DRIVE_WORKSPACE}
arr = !ls {DRIVE_WORKSPACE}
swap = []

def getFolder(arr):
    idx = []
    for elem in arr:
        if elem[:5] == 'train':
            idx.append(int(elem[6:]))
    return idx

if len(arr) == 0: 
    FOLDER_CHECKPOINT = FOLDER_CHECKPOINT + '{}'.format(1) # create 1st train directory
    !cp -a {output_directory}/. {FOLDER_CHECKPOINT} # copy output directory to drive
else:
    for small_arr in arr:
        for elem in small_arr.split():
            swap.append(elem)
    arr = swap

    idx = getFolder(arr)
    print(idx)   
    print('Last trained folder: {}'.format(max(idx)))
    FOLDER_CHECKPOINT = FOLDER_CHECKPOINT + '{}'.format(int(max(idx))+1) # create new train directory
    !cp -a {output_directory}/. {FOLDER_CHECKPOINT} # copy output directory to drive

arr = []
arr = !ls {DRIVE_WORKSPACE}
swap = []

for small_arr in arr:
    for elem in small_arr.split():
        swap.append(elem)
arr = swap
idx = getFolder(arr)
print(idx) 
print('Number of trained folder: {}'.format(len(idx)))
print('New trained folder: {}'.format(max(idx)))

!ls {DRIVE_WORKSPACE}

label_map	 test_img  train_10  train_2  train_4  train_6	train_8
last_checkpoint  train_1   train_11  train_3  train_5  train_7	train_9
[10, 2, 4, 6, 8, 1, 11, 3, 5, 7, 9]
Last trained folder: 11
[1, 12, 4, 7, 10, 2, 5, 8, 11, 3, 6, 9]
Number of trained folder: 12
New trained folder: 12
label_map	 train_1   train_12  train_4  train_7
last_checkpoint  train_10  train_2   train_5  train_8
test_img	 train_11  train_3   train_6  train_9


In [0]:
!test -e {LATEST_CHECKPOINT} && rm -rf {LATEST_CHECKPOINT} || echo LATEST_CHECKPOINT not found && mkdir {LATEST_CHECKPOINT} # create new ckpt
!cd {DRIVE_WORKSPACE} && ls 
!cp -a {output_directory}/. {LATEST_CHECKPOINT}
!cd {LATEST_CHECKPOINT} && ls 

## Inference

# 新しいセクション

In [0]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import glob

%matplotlib inline

In [0]:
!mkdir 'modelInference'
%cd 'modelInference'
!mkdir 'test_images' && cd 'test_images' && pwd
!cd 'test_images' && wget 'https://www.petmd.com/sites/default/files/Acute-Dog-Diarrhea-47066074.jpg'
!cd 'test_images' && wget 'https://cdn-images-1.medium.com/max/1600/1*mONNI1lG9VuiqovpnYqicA.jpeg'
!cd 'test_images' && wget 'https://www.lifewithdogs.tv/wp-content/uploads/2015/01/gfhdgfgfgdfgdf.jpg'

In [0]:
!cp {LATEST_CHECKPOINT}/frozen_inference_graph.pb ./
!cp {PATH_TO_LABELS} ./
!ls

In [0]:
PATH_TO_FROZEN_GRAPH = os.path.join(os.path.abspath('/content/Finetune_SSD_MobileNetV2/models/research/modelInference'), "frozen_inference_graph.pb")
assert os.path.isfile(PATH_TO_FROZEN_GRAPH), '`{}` not exist'.format(PATH_TO_FROZEN_GRAPH)
print(PATH_TO_FROZEN_GRAPH)

PATH_TO_LABELS = os.path.join(os.path.abspath('/content/Finetune_SSD_MobileNetV2/models/research/modelInference'), "pet_label_map.pbtxt")
assert os.path.isfile(PATH_TO_LABELS), '`{}` not exist'.format(PATH_TO_LABELS)
print(PATH_TO_LABELS)

In [0]:
PATH_TO_TEST_IMAGES_DIR = '/content/Finetune_SSD_MobileNetV2/models/research/modelInference/test_images'
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
TEST_IMAGE_PATHS

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)

In [0]:
num_classes = get_num_classes(PATH_TO_LABELS)
num_classes

39

In [0]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    print(image_path)
    print(output_dict['detection_classes'][:5])
    print(output_dict['detection_scores'][:5])
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

In [0]:
!wget http://www.statmt.org/wmt13/training-parallel-commoncrawl.tgz